In [337]:
import pandas as pd
import numpy as np
data = pd.read_excel("Libro2.xlsx")
data.head()

,operador,V1,V2,V3
0,A,570.7,570.5,570.7
1,A,570.7,570.6,570.8
2,A,570.7,570.5,570.7
3,A,570.8,570.6,570.8
4,A,570.8,570.5,570.7


In [338]:
def get_ensayos (data_int,index):
    ops_names= data_int[index].unique().tolist()    
    total_colums = data_int.columns.values.tolist()
    a = set([index])
    b = set(total_colums)
    ensayos = b-a
    ensayos = list(ensayos)
    ensayos.sort()
    return ensayos,ops_names

In [339]:
GE = get_ensayos(data,"operador")
GE

(['V1', 'V2', 'V3'], ['A', 'B', 'C'])

In [251]:
def get_nr(data_int,index):
    n = 0
    r = 0
    transpose_list = data_int[data_int[index] == data_int[index].unique()[0] ][get_ensayos(data_int,index)[0]].values.transpose()
    return transpose_list.shape

# Estudio de Repitibilidad y Reproducibilidad

### Rango de cada parte del equipo
$$R = X_{max} - X_{min}$$

In [317]:
def rango(mediciones): # Obtiene todos los rangos de un determinado operad
    rango_list = list(range(0, len(mediciones)))
    for i in range(len(rango_list)):
        rango_list[i] = max(mediciones[i])-min(mediciones[i])
    print("Rango de cada parte del equipo: "+ str(rango_list))
    return rango_list

### Rango promedio por operador
$$ \overline R = \frac {1}{n} \sum_{i=1}^n R_i $$

In [318]:
def rango_mean(mediciones):
    sum_r = sum(rango(mediciones))
    r_mean = 1/len(mediciones)
    result = r_mean * sum_r
    print("Rango promedio por operador: "+str(result))
    return result

### Rangos promedios totales
$$\overline{\overline R} = \frac {1}{m} \sum_{i=1}^m \overline R_i $$ 

In [319]:
def RPT (data_int,index):
    import numpy as np
    ensayos = get_ensayos(data_int,index)[0]
    ops_names = get_ensayos(data_int,index)[1]
    x=[]
    y=[]
    #Calculamos el rango de cada parte
    for i in ops_names:
        row = data_int[data_int[index]==i][ensayos].values.tolist()
        row = np.array(row)
        x.append(rango_mean(row.transpose()))
    RPT_V = 1/len(ops_names)*sum(x)
    print("Rangos promedios totales:" + str(RPT_V))
    return RPT_V

In [320]:
RPT(data,"operador")

Rango de cada parte del equipo: [5.7000000000000455, 0.20000000000004547, 0.20000000000004547]
Rango promedio por operador: 2.0333333333333785
Rango de cada parte del equipo: [6.5, 0.20000000000004547, 0.20000000000004547]
Rango promedio por operador: 2.30000000000003
Rango de cada parte del equipo: [6.399999999999977, 0.20000000000004547, 0.20000000000004547]
Rango promedio por operador: 2.2666666666666893
Rangos promedios totales:2.200000000000032


2.200000000000032

### Porcentaje de repetibilidad
$$ \%Repetibilidad = \frac{K_1 \overline{\overline R}}{T}*100$$

In [323]:
def PR (data_int, index, Tolerancia):
    K_1 = [4.56,3.05,2.50,2.21]
    ensayos = get_ensayos(data_int,index)[0]
    ops_names = get_ensayos(data_int,index)[1]
    n_med = len(data_int[data_int[index]==ops_names[0]][ensayos[0]])
    temp_num = K_1[n_med-2]*(RPT(data_int,index))
    PR_100 = (temp_num/Tolerancia)*100
    print("Porcentaje de repetibilidad: " + str(PR_100)+"%")
    return PR_100#,K_1[n_med-2],RPT(data_int,index)

In [324]:
PR(data,"operador",2.9)

Rango de cada parte del equipo: [5.7000000000000455, 0.20000000000004547, 0.20000000000004547]
Rango promedio por operador: 2.0333333333333785
Rango de cada parte del equipo: [6.5, 0.20000000000004547, 0.20000000000004547]
Rango promedio por operador: 2.30000000000003
Rango de cada parte del equipo: [6.399999999999977, 0.20000000000004547, 0.20000000000004547]
Rango promedio por operador: 2.2666666666666893
Rangos promedios totales:2.200000000000032
Porcentaje de repetibilidad: 167.65517241379558%


167.65517241379558

### Medición promedio por operador
$$\overline{X_i} = \frac {1}{nr}\sum_{i=0}^n{X_i}$$

In [334]:
def MPPO(data_int,index):
    ensayos = get_ensayos(data_int,index)[0]
    ops_names = get_ensayos(data_int,index)[1]
    temp_sum_x = 0
    n = 0
    r = 0
    xi_sum = []
    for i in ops_names:
        transpose_list = []
        temp_sum_x = 0
        transpose_list = data_int[data_int[index]==i][ensayos].values.transpose().tolist()
        n = len(transpose_list)
        for i in transpose_list:
            r = len(i)
            temp_sum_x += sum(i)
        xi_sum.append(temp_sum_x)
    for i in range(0,len(xi_sum)):
        xi_sum[i] = (1/(n*r))*(xi_sum[i])
    print("Medición promedio por operador: "+str(xi_sum))
    return xi_sum

In [335]:
MPPO(data,"operador")

Medición promedio por operador: [301.3466666666667, 301.4, 301.3]


[301.3466666666667, 301.4, 301.3]

### Diferencia de medición promedio
$$ \overline{X_D} = X_{i_{max}} - X_{i_{max}} $$ 

In [328]:
def DMP (data_int,index):
    var_DMP = MPPO(data_int,index)
    RT = max(var_DMP)-min(var_DMP)
    print("Diferencia de medición promedio: "+str(RT))
    return RT

In [329]:
DMP(data,"operador")

Medición promedio por operador: [301.3466666666667, 301.4, 301.3]
Diferencia de medición promedio: 0.0999999999999659


0.0999999999999659

### Porcentaje de reproducibilidad
$$ \% Reproducibilidad = \frac {\sqrt {
(K_2 * \overline {X_D})^2 - \frac {(K_1 \overline {\overline R})^2}{nr}}}
{T} * 100\%$$

In [343]:
def PREPRO(data_int, index, Tolerancia):
    ops_n= len(data_int[index].unique().tolist())
    nr = get_nr(data_int, index)[0]*get_nr(data_int, index)[1]
    K_1 = [4.56,3.05,2.50,2.21]
    K_2 = [3.65,2.70,2.30,2.08]
    A = (K_2[ops_n-2] * DMP(data_int,index))**2
    B = (K_1[get_nr(data_int, index)[1]-2] * RPT(data_int,index))**2
    C = A - (B/nr)
    if C > 0:
        D = np.sqrt(C)
        R = (D/Tolerancia)*100
        print("Porcentaje de reproducibilidad: "+str(R)+"%")
        return R
    else :
        print("Porcentaje de reproducibilidad: "+str(0)+"%")
        return 0

In [344]:
PREPRO(data,"operador",2.9)

Medición promedio por operador: [570.6733333333332, 570.7466666666667, 570.76]
Diferencia de medición promedio: 0.08666666666681522
Rango de cada parte del equipo: [0.09999999999990905, 0.10000000000002274, 0.09999999999990905]
Rango promedio por operador: 0.09999999999994694
Rango de cada parte del equipo: [0.09999999999990905, 0.10000000000002274, 0.10000000000002274]
Rango promedio por operador: 0.09999999999998484
Rango de cada parte del equipo: [0.09999999999990905, 0.10000000000002274, 0.10000000000002274]
Rango promedio por operador: 0.09999999999998484
Rangos promedios totales:0.0999999999999722
Porcentaje de reproducibilidad: 7.825378188770965%


7.825378188770965

### Porcentaje de la relación entre la repetibilidad y la reproducibilidad
$$\% R\&R =\sqrt{(\%Repetibilidad){^2}+(Reproducibilidad){^2}}$$

In [283]:
def R_R(data_int, index, Tolerancia):
    REPE = PR(data_int, index, Tolerancia)
    REPR = PREPRO(data_int, index, Tolerancia)
    return np.sqrt((REPE**2)+(REPR**2)),REPE,REPR

In [346]:
R_R(data,"operador",11.7)

Rango de cada parte del equipo: [0.09999999999990905, 0.10000000000002274, 0.09999999999990905]
Rango promedio por operador: 0.09999999999994694
Rango de cada parte del equipo: [0.09999999999990905, 0.10000000000002274, 0.10000000000002274]
Rango promedio por operador: 0.09999999999998484
Rango de cada parte del equipo: [0.09999999999990905, 0.10000000000002274, 0.10000000000002274]
Rango promedio por operador: 0.09999999999998484
Rangos promedios totales:0.0999999999999722
Porcentaje de repetibilidad: 1.8888888888883637%
Medición promedio por operador: [570.6733333333332, 570.7466666666667, 570.76]
Diferencia de medición promedio: 0.08666666666681522
Rango de cada parte del equipo: [0.09999999999990905, 0.10000000000002274, 0.09999999999990905]
Rango promedio por operador: 0.09999999999994694
Rango de cada parte del equipo: [0.09999999999990905, 0.10000000000002274, 0.10000000000002274]
Rango promedio por operador: 0.09999999999998484
Rango de cada parte del equipo: [0.099999999999909

(2.7074048740953462, 1.8888888888883637, 1.9396236536269915)